huggingface 데이터를 load하여 colab에서 훈련  
코랩에서 실행하기 위한 다양한 환경설정을 포함  
( drive 연결, cuda설정, 라이브러리 설치 )

In [ ]:
# 코랩에서 실행 시간을 줄이기 위해 전치러된 데이터 load
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip -q install transformers

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip -q install datasets evaluate jiwer accelerate

In [1]:
# model_name_or_path = "openai/whisper-large-v2"
# model_name_or_path ='ymlee/ML_project_voice2text_largev2_1epoch'
model_name_or_path = 'openai/whisper-large-v2'
# task = "transcribe"

language = "Korean" 
language_abbr = 'ko'

In [2]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

/home/yumin/miniconda3/envs/faster/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from transformers import WhisperTokenizer

task = "transcribe"

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language_abbr, task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 원본 데이터 load
from datasets import load_dataset
datasets = load_dataset("Bingsu/zeroth-korean")

In [6]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [7]:
processed_dataset = datasets.map(prepare_dataset, remove_columns=datasets.column_names["train"], num_proc=2)

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, device_map="auto")

/home/yumin/miniconda3/envs/faster/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [11]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [12]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model
config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

In [15]:
!apt-get update

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
!apt-get install cuda-toolkit-11-8

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cuda-toolkit-11-8 is already the newest version (11.8.0-1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [ ]:
!pip -q install bitsandbytes

In [13]:
import os

os.environ["LD_LIBRARY_PATH"] += ":" + "/usr/local/cuda-11/lib64"
os.environ["LD_LIBRARY_PATH"] += ":" + "/usr/local/cuda-11.8/lib64"

model = get_peft_model(model, config)
model.print_trainable_parameters()

model.generation_config.forced_decoder_ids = None

trainable params: 15,728,640 || all params: 1,559,033,600 || trainable%: 1.0089


In [14]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [16]:
import evaluate

metric = evaluate.load("cer")

In [17]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}


In [18]:
 from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./ML_project_voice2text_largev2_fit",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=1,
    # max_steps=8000,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=128,
    # save_steps=1000,
    # eval_steps=1000,
    logging_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="cer",
#    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
    push_to_hub=True,
)

In [21]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home/yumin/miniconda3/envs/faster/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [23]:
trainer.train()

In [24]:
kwargs = {
    "dataset_tags": "Bingsu/zeroth-korean",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: test",
    "language": "ko",
    "model_name": "Whisper large-v2 Korean - ML_project_voice2text_largev2",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-large-v2",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/ymlee/ML_project_voice2text_largev2_1epoch/commit/51165ce3fbe52977f1f2c2c315f5e324060087ee', commit_message='End of training', commit_description='', oid='51165ce3fbe52977f1f2c2c315f5e324060087ee', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('./ML_project_voice2text_largev2_fit')

README.md:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ymlee/ML_project_voice2text_largev2_1epoch/commit/3807bef1d9bdc21d5e250c84197be0768127e176', commit_message='Upload tokenizer', commit_description='', oid='3807bef1d9bdc21d5e250c84197be0768127e176', pr_url=None, pr_revision=None, pr_num=None)